Initial imports

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.stats as si

Sample Data

In [2]:
risk_free_rate = 0.03

implied_vol_df = pd.DataFrame({
    "stock_price": [10, 10, 10, 10, 10, 10, 10, 10, 10],
    "strike_price": [6,7,8,9,10,11,12,13,14],
    "maturity": [0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25],
    "implied_vol": [0.3, 0.29,0.28,0.27,0.26,0.25,0.24,0.23,0.22]
})
implied_vol_df

,stock_price,strike_price,maturity,implied_vol
0,10,6,0.25,0.30
1,10,7,0.25,0.29
2,10,8,0.25,0.28
3,10,9,0.25,0.27
4,10,10,0.25,0.26
5,10,11,0.25,0.25
6,10,12,0.25,0.24
7,10,13,0.25,0.23
8,10,14,0.25,0.22


In [3]:
def black_scholes_call(stock_price, strike_price, maturity, risk_free_rate, implied_vol):
    d1 = (np.log(stock_price/strike_price) + (risk_free_rate + 0.5 * implied_vol**2)*maturity)/(implied_vol*np.sqrt(maturity))
    d2 = (np.log(stock_price/strike_price)+(risk_free_rate-0.5*implied_vol**2)*maturity)/(implied_vol*np.sqrt(maturity))
    return stock_price * si.norm.cdf(d1) - strike_price * np.exp(-risk_free_rate*maturity) * si.norm.cdf(d2)

In [4]:
implied_vol_df["call"] = implied_vol_df.apply(lambda row: black_scholes_call(row["stock_price"], row["strike_price"], row["maturity"], risk_free_rate, row["implied_vol"]), axis=1)
implied_vol_df["call"] = implied_vol_df["call"].round(6)
implied_vol_df

,stock_price,strike_price,maturity,implied_vol,call
0,10,6,0.25,0.30,4.044912
1,10,7,0.25,0.29,3.054635
2,10,8,0.25,0.28,2.085631
3,10,9,0.25,0.27,1.210932
4,10,10,0.25,0.26,0.554541
5,10,11,0.25,0.25,0.185627
6,10,12,0.25,0.24,0.042263
7,10,13,0.25,0.23,0.006087
8,10,14,0.25,0.22,0.000511


In [11]:
projected_df = implied_vol_df
projected_df["call_min"] = implied_vol_df["call"].shift(-1)
projected_df["call_max"] = implied_vol_df["call"]
projected_df["strike_min"] = projected_df["strike_price"]
projected_df["strike_max"] = projected_df["strike_price"].shift(-1)
projected_df["strike_mean"] = (projected_df["strike_min"] + projected_df["strike_max"])/2
projected_df["implied_vol_max"] = implied_vol_df["implied_vol"]
projected_df["implied_vol_min"] = implied_vol_df["implied_vol"].shift(-1)
projected_df["implied_vol_mean"] = (projected_df["implied_vol_min"] + projected_df["implied_vol_max"])/2
projected_df["call_mean"] = projected_df.apply(lambda row: black_scholes_call(row["stock_price"], row['strike_mean'], row['maturity'], risk_free_rate, row['implied_vol_mean']), axis=1)
projected_df = projected_df.drop(["strike_min", "strike_max", "implied_vol_max", "implied_vol_min"], axis=1)
projected_df

,stock_price,strike_price,maturity,implied_vol,call,call_min,call_max,strike_mean,implied_vol_mean,call_mean
0,10,6,0.25,0.30,4.044912,3.054635,4.044912,6.5,0.295,3.549067
1,10,7,0.25,0.29,3.054635,2.085631,3.054635,7.5,0.285,2.564624
2,10,8,0.25,0.28,2.085631,1.210932,2.085631,8.5,0.275,1.629117
3,10,9,0.25,0.27,1.210932,0.554541,1.210932,9.5,0.265,0.848236
4,10,10,0.25,0.26,0.554541,0.185627,0.554541,10.5,0.255,0.335188
5,10,11,0.25,0.25,0.185627,0.042263,0.185627,11.5,0.245,0.093357
6,10,12,0.25,0.24,0.042263,0.006087,0.042263,12.5,0.235,0.017065
7,10,13,0.25,0.23,0.006087,0.000511,0.006087,13.5,0.225,0.001898
8,10,14,0.25,0.22,0.000511,NaN,0.000511,NaN,NaN,NaN


In [ ]:
projected_df['g'] = np.exp(risk_free_rate * projected_df['maturity']) * (projected_df['call_min'] + projected_df['call_max'] - 2*projected_df['call_mean']) / ((projected_df['strike_mean'] - projected_df['strike_price']) ** 2)

projected_df

,stock_price,strike_price,maturity,implied_vol,call,call_min,call_max,strike_mean,implied_vol_mean,call_mean,g
0,10,6,0.25,0.30,4.044912,3.054635,4.044912,6.5,0.295,3.549067,0.005693
1,10,7,0.25,0.29,3.054635,2.085631,3.054635,7.5,0.285,2.564624,0.044401
2,10,8,0.25,0.28,2.085631,1.210932,2.085631,8.5,0.275,1.629117,0.154474
3,10,9,0.25,0.27,1.210932,0.554541,1.210932,9.5,0.265,0.848236,0.278083
4,10,10,0.25,0.26,0.554541,0.185627,0.554541,10.5,0.255,0.335188,0.281270
5,10,11,0.25,0.25,0.185627,0.042263,0.185627,11.5,0.245,0.093357,0.165943
6,10,12,0.25,0.24,0.042263,0.006087,0.042263,12.5,0.235,0.017065,0.057304
7,10,13,0.25,0.23,0.006087,0.000511,0.006087,13.5,0.225,0.001898,0.011291
8,10,14,0.25,0.22,0.000511,NaN,0.000511,NaN,NaN,NaN,NaN


In [15]:
import plotly
import plotly.graph_objects as go
import plotly.express as px